## Data import

We have the tick-by-tick data for 15 currency pairs, given below

    pairs = ['AUDJPY', 'AUDNZD', 'AUDUSD', 'CADJPY', 'CHFJPY', 'EURCHF', 'EURGBP', 'EURJPY',
    'EURUSD', 'GBPJPY', 'GBPUSD', 'NZDUSD', 'USDCAD', 'USDCHF', 'USDJPY']

We download the data for the currency pairs from [TrueFX](https://truefx.com) using the `fetch` script to generate URLs and piping the output to a script that downloads the files from the URLs.

## Data structure

The CSVs downloaded have the following column structure.

    columns = ['name', 'timestamp', 'bid', 'ask']

 * _name_: The name of the currency pair
 * _timestamp_: Timestamp of the tick
 * _bid_: The price at which you can sell in the market
 * _ask_: The price at which you can buy in the market

In [14]:
import pandas as pd
import numpy as np

data_df = pd.read_csv('./AUDJPY-2012-01.csv', header=None)

data_df.columns = ['name', 'tick', 'bid', 'ask']

print(data_df.head())

      name                   tick     bid     ask
0  AUD/JPY  20120102 00:00:00.309  78.714  78.814
1  AUD/JPY  20120102 00:00:00.311  78.716  78.812
2  AUD/JPY  20120102 00:00:05.006  78.716  78.813
3  AUD/JPY  20120102 00:00:05.006  78.721  78.811
4  AUD/JPY  20120102 00:00:05.007  78.726  78.811


## Data preparation and feature extraction

1. Resampling: Resample the data into 15-minute intervals.
2. Add features: OHLCV (Open, High, Low, Close, Volume) for the intervals.


The prepared data has columns:

    columns = ['tick_start', 'tick_end', 'open', 'high', 'low', 'close', 'volume']

In [15]:
# Let the dataframe know the timestamp column

data_df['tick'] = pd.to_datetime(data_df['tick'])

In [23]:
print(data_df.head())

      name                    tick     bid     ask
0  AUD/JPY 2012-01-02 00:00:00.309  78.714  78.814
1  AUD/JPY 2012-01-02 00:00:00.311  78.716  78.812
2  AUD/JPY 2012-01-02 00:00:05.006  78.716  78.813
3  AUD/JPY 2012-01-02 00:00:05.006  78.721  78.811
4  AUD/JPY 2012-01-02 00:00:05.007  78.726  78.811


In [26]:
# Drop unnecesarry column
del data_df['name']

In [43]:
last_time = data_df['tick'][0]
cur_price = data_df['ask'][0]

# Each 15 minute interval is represented by the following variables.
cur_interval = {'tick_start': last_time,
                'tick_end': last_time,
                'open': cur_price,
                'high': cur_price,
                'low': cur_price,
                'close': cur_price,
                'volume': 0}
intervals = []

# Sample the intervals from the raw data.
for row in data_df.itertuples():
    cur_time = row.tick
    cur_price = row.ask
    if cur_time.timestamp() - last_time.timestamp() > 3600*24:
        # Append this to the list of intervals.
        intervals.append(cur_interval.copy())
        
        cur_interval = {'tick_start':cur_time,
                'tick_end': cur_time,
                'open': cur_price,
                'high': cur_price,
                'low': cur_price,
                'close': cur_price,
                'volume': 1}
        last_time = cur_time
    else:
        cur_interval['tick_end'] = cur_time
        cur_interval['close'] = cur_price
        cur_interval['high'] = max(cur_price, cur_interval['high'])
        cur_interval['low'] = min(cur_price, cur_interval['low'])
        cur_interval['volume'] += 1

        
# The last remaining (incomplete) interval.
intervals.append(cur_interval.copy())

# Create a dataframe.
processed_data = pd.DataFrame(intervals)

print(processed_data.head())
print(processed_data.describe())

processed_data.to_csv('AUDJPY-2012-01_daily.csv', index=False)

    close    high     low    open                tick_end  \
0  78.888  78.902  78.474  78.814 2012-01-02 23:59:59.540   
1  79.575  79.745  78.887  78.889 2012-01-04 00:00:02.235   
2  79.326  79.593  79.103  79.575 2012-01-05 00:00:04.948   
3  79.118  79.619  78.699  79.329 2012-01-06 00:00:05.010   
4  78.751  79.504  78.660  79.115 2012-01-06 21:59:55.307   

               tick_start  volume  
0 2012-01-02 00:00:00.309   27727  
1 2012-01-03 00:00:04.295  131203  
2 2012-01-04 00:00:05.011  155614  
3 2012-01-05 00:00:05.059  209396  
4 2012-01-06 00:00:05.087  156850  
           close       high       low       open        volume
count  25.000000  25.000000  25.00000  25.000000      25.00000
mean   80.200560  80.460680  79.74280  80.065840  126954.92000
std     1.156093   1.201581   1.13875   1.166029   66953.82799
min    78.685000  78.810000  78.10100  78.407000    1021.00000
25%    79.270000  79.508000  78.81300  79.264000   94934.00000
50%    79.758000  79.870000  79.32900  

In [44]:
s = pd.Series(pd.to_datetime(np.arange(100000), unit='ms'))
%timeit s - s.shift()

1000 loops, best of 3: 846 µs per loop
